In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Reading in the input data from a csv file
df = pd.read_csv('loan_data.csv', sep = ';')
df

,ID,Quarter,State_faze,Exposure at initiation,Payment,Remaining exposure,Recovered exposure after default
0,1,1,0-15,80141.246320,8014.12,72127.121690,0.000000
1,2,1,0-15,82195.077490,8219.51,73975.569740,0.000000
2,3,1,0-15,31057.584550,3105.76,27951.826100,0.000000
3,4,1,0-15,4565.938116,456.59,4109.344304,0.000000
4,5,1,0-15,63041.144870,6304.11,56737.030390,0.000000
...,...,...,...,...,...,...,...
5591,275,20,def,97124.023430,9712.40,87411.621090,25666.161680
5592,276,20,def,50067.836670,5006.78,20027.134670,8623.545586
5593,277,20,def,79596.339130,7959.63,63677.071310,5108.167798
5594,278,20,def,93198.428820,9319.84,74558.743060,19438.721890


In [4]:
#Estimating the quarterly transition matrix
df_sorted = df.sort_values(by=["ID", "Quarter"])
df_sorted["Next_State"] = df_sorted.groupby("ID")["State_faze"].shift(-1)
transitions = df_sorted.dropna(subset=["Next_State"])

quarterly_matrix = pd.crosstab(
    index=transitions["State_faze"],
    columns=transitions["Next_State"],
    normalize=0
)
quarterly_matrix

Next_State,0-15,15-30,30-60,60-90,def,paid
State_faze,,,,,,
0-15,0.762887,0.092784,0.076583,0.021355,0.013255,0.033137
15-30,0.096875,0.493750,0.184375,0.159375,0.065625,0.000000
30-60,0.038869,0.102473,0.307420,0.282686,0.268551,0.000000
60-90,0.000000,0.029915,0.141026,0.320513,0.508547,0.000000
def,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
paid,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [5]:
#Estimating the annual transition matrix
df_sorted = df.sort_values(by=["ID", "Quarter"])
df_sorted["Next_State"] = df_sorted.groupby("ID")["State_faze"].shift(-4)
transitions = df_sorted.dropna(subset=["Next_State"])

annual_matrix = pd.crosstab(
    index=transitions["State_faze"],
    columns=transitions["Next_State"],
    normalize=0
)
annual_matrix

Next_State,0-15,15-30,30-60,60-90,def,paid
State_faze,,,,,,
0-15,0.352071,0.112426,0.101331,0.090237,0.222633,0.121302
15-30,0.114286,0.146032,0.088889,0.098413,0.526984,0.025397
30-60,0.035461,0.060284,0.081560,0.095745,0.712766,0.014184
60-90,0.004405,0.035242,0.026432,0.044053,0.889868,0.000000
def,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
paid,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [6]:
#Filtering the data and calculating the mean recovery rate for defaults
defaults = df.loc[df['Recovered exposure after default'] > 0]
defaults['RecoveryRate'] = defaults['Recovered exposure after default'] / defaults['Remaining exposure']
defaults

,ID,Quarter,State_faze,Exposure at initiation,Payment,Remaining exposure,Recovered exposure after default,RecoveryRate
325,46,2,def,25848.35932,2584.84,23263.52339,1888.527341,0.081180
337,58,2,def,32457.24749,3245.72,29211.52274,21710.331430,0.743211
405,126,2,def,48732.20361,4873.22,43858.98325,7244.744825,0.165183
462,183,2,def,38932.25681,3893.23,35039.03113,28002.468000,0.799179
537,258,2,def,86753.76837,8675.38,78078.39154,41205.609900,0.527747
...,...,...,...,...,...,...,...,...
5591,275,20,def,97124.02343,9712.40,87411.62109,25666.161680,0.293624
5592,276,20,def,50067.83667,5006.78,20027.13467,8623.545586,0.430593
5593,277,20,def,79596.33913,7959.63,63677.07131,5108.167798,0.080220
5594,278,20,def,93198.42882,9319.84,74558.74306,19438.721890,0.260717


In [7]:
defaults['RecoveryRate'].mean()

0.4878994099098422

In [8]:
#Importing the new loans
new_loans = pd.read_excel('Data_loans.xlsx', sheet_name='New_data')
new_loans

,ID,State,Current exposure
0,281,0-15,62408.534372
1,282,60-90,26572.805359
2,283,0-15,11305.692170
3,284,0-15,82883.125607
4,285,15-30,49381.383725
...,...,...,...
195,476,0-15,79914.461639
196,477,0-15,56717.583380
197,478,0-15,5326.052543
198,479,0-15,95438.593457


In [9]:
#Calculating the Expected Credit Loss for new loans
new_loans['PD'] = new_loans['State'].map(annual_matrix['def'])
new_loans['ECL'] = new_loans['Current exposure'] * new_loans['PD'] * (1 - defaults['RecoveryRate'].mean())
new_loans

,ID,State,Current exposure,PD,ECL
0,281,0-15,62408.534372,0.222633,7115.231976
1,282,60-90,26572.805359,0.889868,12109.276474
2,283,0-15,11305.692170,0.222633,1288.968300
3,284,0-15,82883.125607,0.222633,9449.551596
4,285,15-30,49381.383725,0.526984,13326.498837
...,...,...,...,...,...
195,476,0-15,79914.461639,0.222633,9111.092553
196,477,0-15,56717.583380,0.222633,6466.403464
197,478,0-15,5326.052543,0.222633,607.226235
198,479,0-15,95438.593457,0.222633,10881.007521


In [12]:
#Estimating ECL for the whole portfolio
new_loans['ECL'].sum()

2247170.2607052606